<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5cdb611ad3f46e26a77035ca7e6dec1545de522df54784a1bc0d315e5c8fa50d
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-10 14:29:49
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -23.35 K (-0.17%)
Current PnL: -29.34 L (-18.95%)
CY Booked + Current PnL: -15.05 L (-9.72%)
-------------------
Total profit:  1.21 L
Total loss:  -30.55 L
-------------------
Total Booked + Current PnL: 11.93 L (9.33%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.29%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.97 L (67.63%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.81,-17.37,21.13,0.09,17596.0,-17500.0,83277.0,93.38,37.0,M-SC,2.35,253.0,-0.99,0.62,9.41,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,2.32,-12.71,14.58,0.02,22216.0,-22188.0,152371.0,132.14,52.0,M-SC,16.62,234.0,-1.00,1.14,32.54,OX40N,NTT,PAINTS
50,MASFIN,398.61,0.71,-2.25,24.86,22.05,23810.0,-2205.0,95775.0,-15.54,55.0,H-SC,6.86,164.0,-0.09,0.71,39.22,XR,ATH,FINANCE
43,ITC,452.00,0.45,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,46.0,X-LC,1.72,5.0,-0.13,1.47,3.90,X40,NTT,FMCG
82,VOLTAS,1530.00,0.10,4.61,14.42,19.69,28924.0,8838.0,200580.0,-5.64,44.0,X-MC,2.56,78.0,0.31,1.50,11.72,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.0,3.18,-44.61,144.20,35.27,90700.0,-50650.0,62899.0,-57.75,30.0,L-SC,10.20,270.0,-0.56,0.47,53.95,XR,NTT,HOTELS
36,ICICIPRULI,790.0,3.14,6.62,22.83,30.97,43022.0,11705.0,188443.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.41,20.02,X40,ATH,INSURANCE
44,JCHAC,2282.0,3.03,-36.02,56.28,-0.01,46021.0,-46034.0,81771.0,16639.13,24.0,M-SC,4.11,233.0,-1.00,0.61,4.66,OX40N,NTT,AC
80,VALIANTORG,838.0,2.49,-43.15,206.34,74.15,191911.0,-70598.0,93007.0,-267.19,49.0,H-SC,21.54,149.0,-0.37,0.69,19.22,XR,NTT,CHEMICALS
6,ASIANTILES,137.0,2.36,-17.65,120.61,81.67,93174.0,-16558.0,77252.0,6938.89,40.0,L-SC,14.30,271.0,-0.18,0.58,49.60,XR,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-6.33,-18.69,49.53,21.58,56439.0,-26190.0,113949.0,-59.02,18.0,X-MC,6.26,56.0,-0.46,0.85,0.88,X40N,ATH,IT
57,RAJOOENG,143.10,-2.77,-37.40,123.00,39.61,78929.0,-38330.0,64170.0,-57.57,23.0,H-SC,26.27,135.0,-0.49,0.48,0.00,AR,ATH,MISC
61,ROUTE,2227.21,-2.68,-48.91,230.91,69.05,156968.0,-65088.0,67978.0,-58.67,46.0,H-SC,23.21,140.0,-0.41,0.51,4.84,SR,ATH,IT
74,TITAGARH,1548.00,-2.43,-28.92,105.02,45.74,180793.0,-70026.0,172151.0,-5.35,26.0,H-SC,4.04,174.0,-0.39,1.28,11.97,XY24,NTT,ENGINEERING
23,DMART,5391.45,-2.41,-9.32,41.29,28.12,78779.0,-19612.0,190795.0,-21.75,30.0,X-LC,4.64,19.0,-0.25,1.42,12.10,X40N,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,1.15,-6.93,123.40,107.91,169561.0,-10236.0,137408.0,-24.20,37.0,M-SC,12.18,216.0,-0.06,1.03,2.45,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.32,-12.71,14.58,0.02,22216.0,-22188.0,152371.0,132.14,52.0,M-SC,16.62,234.0,-1.00,1.14,32.54,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,0.81,-17.37,21.13,0.09,17596.0,-17500.0,83277.0,93.38,37.0,M-SC,2.35,253.0,-0.99,0.62,9.41,OX40N,NTT,DURABLES
66,SIS,528.00,0.27,-24.45,60.51,21.26,50757.0,-27150.0,83882.0,1978.76,47.0,H-SC,4.70,166.0,-0.53,0.63,13.33,OX40N,NTT,MISC
47,KANSAINER,340.00,1.13,-25.10,51.50,13.47,104019.0,-67689.0,201978.0,-69.28,31.0,H-SC,3.65,158.0,-0.65,1.51,2.70,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.36,5.96,62.58,72.27,112356.0,10091.0,179539.0,-9.26,65.0,M-LC,2.08,99.0,0.09,1.34,12.5,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.36,5.96,62.58,72.27,112356.0,10091.0,179539.0,-9.26,65.0,M-LC,2.08,99.0,0.09,1.34,12.50,XR,NTT,IT
38,INDIAMART,4810.62,-1.65,-4.58,116.65,106.72,137281.0,-5650.0,117686.0,-52.73,30.0,H-SC,1.60,139.0,-0.04,0.88,17.73,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.56,-2.41,38.98,35.63,79723.0,-5050.0,204524.0,-16.30,39.0,H-MC,3.16,119.0,-0.06,1.53,13.86,AR,ATH,PHARMA
25,FINCABLES,1641.55,1.15,-6.93,123.40,107.91,169561.0,-10236.0,137408.0,-24.20,37.0,M-SC,12.18,216.0,-0.06,1.03,2.45,OX40N,ATH,CABLES
37,IEX,219.00,-0.81,-4.95,56.37,48.63,107130.0,-9906.0,190048.0,-36.71,45.0,H-SC,14.08,136.0,-0.09,1.42,6.03,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,DIXON,18931.72,-6.33,-18.69,49.53,21.58,56439.0,-26190.0,113949.0,-59.02,18.0,X-MC,6.26,56.0,-0.46,0.85,0.88,X40N,ATH,IT
9,BAJAJHFL,181.50,0.48,-21.49,89.08,48.45,140233.0,-43082.0,157424.0,-27.18,24.0,X-MC,11.69,64.0,-0.31,1.17,2.50,X40N,ATH,FINANCE
75,TMPV,600.00,-0.51,-18.73,74.95,42.18,168362.0,-51778.0,224632.0,-26.74,24.0,X-LC,5.63,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
53,PAGEIND,51605.08,-0.74,-9.81,39.78,26.07,58747.0,-16060.0,147680.0,-33.64,28.0,X-MC,11.87,55.0,-0.27,1.10,0.00,X40,ATH,APPARELS
33,HINDUNILVR,2922.00,-0.23,-10.48,26.97,13.67,62066.0,-26933.0,230130.0,-17.11,29.0,X-LC,2.79,14.0,-0.43,1.72,8.64,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,0.45,-21.78,96.24,53.50,227153.0,-65723.0,236028.0,-62.10,33.0,X-MC,1.34,58.0,-0.29,1.76,4.84,XY24,NTT,FMCG
15,CAMS,4762.0,-1.35,-80.62,544.04,24.79,265492.0,-203056.0,48800.0,-81.54,37.0,X-SC,1.50,86.0,-0.76,0.36,19.30,X40N,NTT,MISC
43,ITC,452.0,0.45,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,46.0,X-LC,1.72,5.0,-0.13,1.47,3.90,X40,NTT,FMCG
36,ICICIPRULI,790.0,3.14,6.62,22.83,30.97,43022.0,11705.0,188443.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.41,20.02,X40,ATH,INSURANCE
3,ACC,3906.0,-0.12,-25.08,119.09,64.15,212314.0,-59671.0,178280.0,-56.31,35.0,X-SC,1.98,82.0,-0.28,1.33,0.01,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HAVELLS,2069.16,-1.46,-10.43,47.90,32.48,140725.0,-34201.0,293790.0,-14.91,36.0,X-MC,5.31,67.0,-0.24,2.19,0.00,X40,ATH,ELECTRICAL
53,PAGEIND,51605.08,-0.74,-9.81,39.78,26.07,58747.0,-16060.0,147680.0,-33.64,28.0,X-MC,11.87,55.0,-0.27,1.10,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-0.51,-18.73,74.95,42.18,168362.0,-51778.0,224632.0,-26.74,24.0,X-LC,5.63,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
3,ACC,3906.00,-0.12,-25.08,119.09,64.15,212314.0,-59671.0,178280.0,-56.31,35.0,X-SC,1.98,82.0,-0.28,1.33,0.01,XY24,BTT,CEMENT
54,PGHH,17907.65,0.02,-5.89,42.12,33.76,79607.0,-11820.0,189000.0,-33.58,34.0,X-MC,5.15,57.0,-0.15,1.41,0.10,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.09,-29.90,103.77,42.84,47288.0,-19436.0,45570.0,-54.27,41.0,X-SC,36.81,83.0,-0.41,0.34,3.10,XY24,NTT,MISC
15,CAMS,4762.00,-1.35,-80.62,544.04,24.79,265492.0,-203056.0,48800.0,-81.54,37.0,X-SC,1.50,86.0,-0.76,0.36,19.30,X40N,NTT,MISC
59,RELAXO,1176.00,-2.23,-46.71,185.78,52.31,143723.0,-67798.0,77362.0,-43.28,46.0,X-SC,6.16,91.0,-0.47,0.58,3.55,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.08,-36.46,112.79,35.20,92211.0,-46915.0,81755.0,7.15,40.0,X-SC,15.40,92.0,-0.51,0.61,3.85,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-6.33,-18.69,49.53,21.58,56439.0,-26190.0,113949.0,-59.02,18.0,X-MC,6.26,56.0,-0.46,0.85,0.88,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-0.69,-13.43,37.78,19.28,113148.0,-46463.0,299493.0,-24.60,59.0,X-LC,5.49,1.0,-0.41,2.24,10.72,X40,ATH,IT
41,INFY,2275.00,-0.73,8.85,42.28,54.87,144000.0,27699.0,340587.0,-15.29,64.0,X-LC,3.38,2.0,0.19,2.54,17.87,X40,BTT,IT
75,TMPV,600.00,-0.51,-18.73,74.95,42.18,168362.0,-51778.0,224632.0,-26.74,24.0,X-LC,5.63,3.0,-0.31,1.68,0.00,XY24,NTT,AUTO
81,VBL,671.64,0.28,-4.48,42.04,35.67,126825.0,-14164.0,301678.0,-15.77,54.0,X-LC,4.73,4.0,-0.11,2.25,8.75,X40N,ATH,FMCG
43,ITC,452.00,0.45,-1.57,12.20,10.44,24033.0,-3144.0,196994.0,-41.15,46.0,X-LC,1.72,5.0,-0.13,1.47,3.90,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,2.36,-17.65,120.61,81.67,93174.0,-16558.0,77252.0,6938.89,40.0,L-SC,14.30,271.0,-0.18,0.58,49.60,XR,NTT,CERAMICS
50,MASFIN,398.61,0.71,-2.25,24.86,22.05,23810.0,-2205.0,95775.0,-15.54,55.0,H-SC,6.86,164.0,-0.09,0.71,39.22,XR,ATH,FINANCE
13,BSOFT,831.70,-1.86,-23.42,96.90,50.79,104373.0,-32941.0,107712.0,-1.82,64.0,H-SC,5.80,151.0,-0.32,0.80,25.41,XR,ATH,IT
39,INDIGOPNTS,1408.00,2.32,-12.71,14.58,0.02,22216.0,-22188.0,152371.0,132.14,52.0,M-SC,16.62,234.0,-1.00,1.14,32.54,OX40N,NTT,PAINTS
36,ICICIPRULI,790.00,3.14,6.62,22.83,30.97,43022.0,11705.0,188443.0,-18.51,64.0,X-MC,1.97,75.0,0.27,1.41,20.02,X40,ATH,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.86,-23.42,96.90,50.79,104373.0,-32941.0,107712.0,-1.82,64.0,H-SC,5.80,151.0,-0.32,0.80,25.41,XR,ATH,IT
32,HCLTECH,1922.01,0.19,7.78,15.73,24.73,41015.0,18822.0,260746.0,12.34,62.0,X-LC,5.96,7.0,0.46,1.95,24.20,X40,ATH,IT
84,WIPRO,420.00,0.36,5.96,62.58,72.27,112356.0,10091.0,179539.0,-9.26,65.0,M-LC,2.08,99.0,0.09,1.34,12.50,XR,NTT,IT
56,RAJESHEXPO,518.00,0.71,-63.34,172.98,0.07,86985.0,-86891.0,50286.0,1743.43,55.0,L-SC,2.30,268.0,-1.00,0.38,25.06,OX40N,NTT,JEWELLERY
39,INDIGOPNTS,1408.00,2.32,-12.71,14.58,0.02,22216.0,-22188.0,152371.0,132.14,52.0,M-SC,16.62,234.0,-1.00,1.14,32.54,OX40N,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.75
1,25,45.77
2,50,76.70


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.02
MC    30.34
LC    24.65
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.28
X40      22.52
X40N     10.83
XR       10.08
XY25      9.39
AR        9.32
OX40N     7.82
SR        1.03
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.45
X-MC    23.57
X-LC    21.08
M-SC    12.15
X-SC     6.99
H-MC     5.00
L-SC     1.43
M-MC     1.41
M-LC     1.34
H-LC     1.18
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.39,-7.99,42.91
IT,12.75,-17.98,81.77
FINANCE,9.50,-21.65,73.05
ELECTRICAL,6.24,-12.29,53.41
PAINTS,6.14,-12.27,28.95
MISC,6.08,-59.60,124.99
INSURANCE,4.66,-1.43,37.17
PHARMA,4.12,-1.26,33.67
AUTO,3.39,-26.35,80.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3246684.0,21
AR,1323780.0,10
XR,1310357.0,13
X40,1021490.0,14
X40N,995931.0,9
OX40N,745261.0,10
XY25,384875.0,6
SR,283326.0,2
MH,85221.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3701439.0,25
M-SC,1430626.0,15
X-MC,1323550.0,16
X-SC,982733.0,8
X-LC,939859.0,11
H-MC,409462.0,3
L-SC,270859.0,3
M-LC,112356.0,1
H-LC,75527.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1295133.0      6
           AR         905461.0      5
M-SC       XY24       815909.0      6
H-SC       XR         785004.0      7
X-SC       X40N       505891.0      3
X-MC       X40        458176.0      7
           XY24       393347.0      3
X-LC       X40        380318.0      5
H-SC       OX40N      347294.0      4
M-SC       OX40N      310982.0      5
X-SC       XY24       293846.0      3
X-MC       X40N       284436.0      4
H-SC       SR         283326.0      2
X-LC       XY24       252250.0      2
H-MC       AR         213263.0      2
X-LC       X40N       205604.0      2
H-MC       XY24       196199.0      1
X-MC       XY25       187591.0      2
L-SC       XR         183874.0      2
X-SC       X40        182996.0      2
M-SC       XR         174206.0      2
           AR         129529.0      2
M-LC       XR         112356.0      1
X-LC       XY25       101687.0      2
L-SC       OX40N       86985.0      1
H-SC       MH          85221.0      1
H-LC       AR          75527.0      1
M-MC       XY25        56563.0      1
L-MC       XR          54917.0      1
L-LC       XY25        39034.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
